#### Visualize census tract data

In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import glob

In [10]:
# find census outputs
csv_outputs = glob.glob("../census_outputs/*.csv")

# load census outputs as dataframe
outputs_df_lst = [ pd.read_csv(file) for file in csv_outputs ]

census_tract_df = pd.concat(outputs_df_lst)

In [12]:
census_tract_df.measurement.unique()

array(['estimate', 'annotation of estimate', 'margin of error',
       'annotation of margin of error', 'percent',
       'annotation of percent', 'percent margin of error',
       'annotation of percent margin of error'], dtype=object)